In [2]:
#Takes the peaks and stores them as fingerprints
import numpy as np

In [14]:
import pickle
def peaks_to_finger_newsongs(local_peaks):
    with open("songs.pkl", mode="rb") as opened_file:
        song_dic = pickle.load(opened_file)
    freq, time= np.where(local_peaks)
    song_name=input("Song Name= ")
    for i in range(len(local_peaks)-15):
        j=1
        while j<16:
            finger=(freq[i], freq[i+j], (time[j]-time[i]))
            j+=1
            song_dic.update([finger, (song_name, time[i])])
    with open("songs.pkl", mode="wb") as opened_file:
        pickle.dump(song_dic, opened_file)

In [9]:
#Testing
local_peaks=np.array([True, True])
peaks_to_finger_newsongs(local_peaks)

Song Name= Despacito


'Despacito'